In [2]:
# 使用Dask（超大规模数据）
import dask.dataframe as dd
ddf = dd.read_parquet("test-00000-of-00001.parquet", engine="pyarrow")
result = ddf.compute()
result

,tokens,ner_tags
0,"[一, 节, 课, 的, 时, 间, 真, 心, 感, 动, 了, 李, 开, 复, 感, 动]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 14, 14, ..."
1,"[回, 复, 支, 持, ，, 赞, 成, ，, 哈, 哈, 米, 八, 吴, 够, 历, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[剑, 网, 乱, 世, 长, 安, 公, 测, 盛, 典, 今, 日, 开, 启, ，, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[在, 街, 上, 听, 见, 音, 乐, 我, 舞, 动, 起, 来, 很, 丢, 人, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[三, 毛, 说, 我, 唯, 一, 锲, 而, 不, 舍, ，, 愿, 意, 以, 自, ...","[13, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...
265,"[么, 儿, ！, [, 爱, 你, ], /, /, @, 微, 博, 搞, 笑, 排, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
266,"[感, 恩, 大, 回, 馈, ，, 孟, 小, 帅, 沈, 阳, 店, 专, 属, 活, ...","[0, 0, 0, 0, 0, 0, 9, 10, 10, 10, 10, 10, 0, 0..."
267,"[我, 叫, 赵, 新, 民, 我, 也, 要, 吃, 斤, 刚, 刚, 好, 呢, 肉, ...","[0, 0, 13, 14, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
268,"[#, 微, 博, 益, 起, 来, #, 努, 力, 加, 油, O, (, ∩, _, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
import spacy
# 处理文本
nlp = spacy.load('zh_core_web_sm')
doc = nlp("""
#计算机科学1. 哈希表-哈希表（Hash Table）是一种数据结构，通过键值对存储数据。- 它使用哈希函数计算键的索引，实现O(1)时间复杂度的查找。- 冲突解决方法：链地址法（Chaining）、开放寻址法（Open Addressing）。2. 深度学习- 神经网络由输入层、隐藏层、输出层组成。- 反向传播用于优化模型参数。- 常见框架：PyTorch、TensorFlow。# 数学1. 贝叶斯定理- P(A|B) = P(B|A) * P(A) / P(B)- 用于概率推断，如垃圾邮件分类。2. 线性代数- 矩阵乘法不满足交换律：A×B ≠ B×A。- 特征值和特征向量：Av = λv。
""")

# 遍历识别出的实体
for ent in doc.ents:
    # 打印实体文本及其标注
    print(ent.text, ent.label_)

1. 哈希表 QUANTITY
哈希表 PERSON
Hash Table PERSON
数计算键 CARDINAL
1 CARDINAL
2 CARDINAL
1. CARDINAL
贝叶斯 GPE
2 CARDINAL


In [7]:
def convert_to_entity_list(tokens, ner_tags):
    entities = []
    current_entity = []
    current_b_tag = None

    for token, tag in zip(tokens, ner_tags):
        if tag == 0:  # O 标签
            if current_entity:
                entities.append("".join(current_entity))
                current_entity = []
                current_b_tag = None
        elif tag % 2 == 1:  # B 标签 (奇数)
            if current_entity:
                entities.append("".join(current_entity))
            current_entity = [token]
            current_b_tag = tag
        elif tag % 2 == 0:  # I 标签 (偶数)
            if current_b_tag is not None and tag == current_b_tag + 1:
                current_entity.append(token)

    if current_entity:
        entities.append("".join(current_entity))

    return entities

# 示例使用
tokens = ["这","次","代","表","大","会","是","在","中","国","改","革","开","放","和","社","会","主","义","现","代","化","建","设","发","展","的","关","键","时","刻","召","开","的","历","史","性","会","议","。"]
ner_tags = [0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

x = "".join(tokens)  # "这次代表大会是在中国改革开放和社会主义现代化建设发展的关键时刻召开的历史性会议。"
y = convert_to_entity_list(tokens, ner_tags)  # ["中国"]
x,y

('这次代表大会是在中国改革开放和社会主义现代化建设发展的关键时刻召开的历史性会议。', ['中国'])

In [12]:
import dask.dataframe as dd
import pandas as pd
import pyarrow as pa
from dask.diagnostics import ProgressBar

def convert_to_entity_list(row):
    """
    将单行数据转换为实体列表
    输入: 包含tokens和ner_tags的DataFrame行
    输出: (完整文本, 实体列表) 的元组
    """
    tokens = row['tokens']
    ner_tags = row['ner_tags']

    entities = []
    current_entity = []
    current_b_tag = None

    try:
        for token, tag in zip(tokens, ner_tags):
            if tag == 0:  # O标签
                if current_entity:
                    entities.append("".join(current_entity))
                    current_entity = []
                    current_b_tag = None
            elif tag % 2 == 1:  # B标签 (实体起始)
                if current_entity:
                    entities.append("".join(current_entity))
                current_entity = [token]
                current_b_tag = tag
            elif tag % 2 == 0:  # I标签 (实体内部)
                if current_b_tag is not None and tag == current_b_tag + 1:
                    current_entity.append(token)

        if current_entity:
            entities.append("".join(current_entity))

    except Exception as e:
        print(f"实体提取失败: {e}")
        entities = []  # 错误时返回空列表

    # 将分词列表合并为完整文本
    full_text = "".join(tokens)

    return pd.Series({
        'x': full_text,
        'y': entities  # 保持为列表类型
    })

# ===== 主处理流程 =====
if __name__ == "__main__":
    # 1. 读取原始Parquet文件
    ddf = dd.read_parquet(
        "test-00000-of-00001.parquet",
        engine="pyarrow",
        blocksize="128MB",  # 优化分块大小[1,3](@ref)
        columns=['tokens', 'ner_tags']  # 仅加载必要列[2](@ref)
    )

    # 2. 应用转换函数（显示进度条）
    with ProgressBar():  # 添加进度监控[3](@ref)
        new_ddf = ddf.map_partitions(
            lambda df: df.apply(convert_to_entity_list, axis=1),
            meta={'x': 'str', 'y': 'object'}  # 指定输出类型[5](@ref)
        )

    # 3. 定义明确的Parquet schema（关键修复）[1,4,7](@ref)
    schema = pa.schema([
        ('x', pa.string()),          # 文本列：字符串类型
        ('y', pa.list_(pa.string())) # 实体列：字符串列表类型
    ])

    # 4. 保存为新的Parquet文件（显式指定schema）
    new_ddf.to_parquet(
        "processed_dataset",
        engine="pyarrow",
        schema=schema,  # 显式定义schema解决类型错误[5,7](@ref)
        compression="snappy",  # 优化压缩格式[1,2](@ref)
        write_index=False,
        overwrite=True,
        compute_kwargs={'scheduler': 'threads'}  # 多线程写入加速[3](@ref)
    )

    print("处理完成！输出保存在 processed_dataset 目录")

处理完成！输出保存在 processed_dataset 目录


In [14]:
# 使用Dask（超大规模数据）
import dask.dataframe as dd
ddf = dd.read_parquet("processed_dataset/part.0.parquet", engine="pyarrow")
result = ddf.compute()
result

,x,y
0,一节课的时间真心感动了李开复感动,[李开复]
1,回复支持，赞成，哈哈米八吴够历史要的陈小奥丁丁我爱小肥肥一族大头仔大家团结一致，誓要去台湾饮...,"[陈小奥, 台湾]"
2,剑网乱世长安公测盛典今日开启，海量豪礼火爆开送精美挂件、听雨·汉服娃娃、诙谐双骑独轮车等你来...,[]
3,在街上听见音乐我舞动起来很丢人？真的很丢人吗？,[]
4,三毛说我唯一锲而不舍，愿意以自己的生命去努力的，只不过是保守我个人的心怀意念，在我有生之日，...,[三毛]
...,...,...
265,么儿！[爱你]//@微博搞笑排行榜:想要这个蛋糕[花心][花心],[]
266,感恩大回馈，孟小帅沈阳店专属活动http://t.cn/Rhnxj3A,[孟小帅沈阳店]
267,我叫赵新民我也要吃斤刚刚好呢肉包纸？这货竟然是肉包纸太萌了吧伦家都不舍得吃啦喜欢吃，就关注吃...,"[赵新民, 吃货]"
268,#微博益起来#努力加油O(∩_∩)O我在:http://t.cn/8khf0QM,[]
